https://www.kaggle.com/laxmikantnishad/covid-19-2-para

In [1]:
is_local=False

In [2]:
if not is_local:
    !conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
    !conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
    !conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
    !conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
    !conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
    !conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y
#     !pip install pylibjpeg 
    #pylibjpeg-libjpeg


######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - \ done

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - \ | done

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - done

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - \ | / - \ | / - \ | / - \ | / - \ | done

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - done

#########################

In [3]:
import os

from PIL import Image
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

（1）dcm to .png

In [4]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    import pydicom
    from pydicom.pixel_data_handlers.util import apply_voi_lut
#     import pylibjpeg
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data
        

In [5]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [6]:
def prepare_test_images(img_size=512, split = 'test'):
    save_dir = f'./{split}/'

    os.makedirs(save_dir, exist_ok=True)

    save_dir = f'./{split}/study/'
    os.makedirs(save_dir, exist_ok=True)

    for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=img_size)  
            study = dirname.split('/')[-2] + '_study.png'
            #study = dirname.split('/')[-2] + '_study'
            im.save(os.path.join(save_dir, study))

In [7]:
!ls ./test/study

ls: cannot access './test/study': No such file or directory


In [8]:
import sys; 

package_paths = [
    '../input/image-fmix/FMix-master'                            #FMix是一种数据增强方法（最近比较火的一种）
]

if not is_local:
    package_paths.append('../input/pytorch-image-models/pytorch-image-models-master') #导入pytorch模型

for pth in package_paths:
    sys.path.append(pth)

In [9]:
from glob import glob
import cv2
from skimage import io
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.cuda.amp import autocast, GradScaler
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

import timm
import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
from scipy.ndimage.interpolation import zoom
# import pydicom


from fmix import sample_mask, make_low_freq_image, binarise_mask
from sklearn.model_selection import GroupKFold, StratifiedKFold

# import pydicom
# from pydicom.pixel_data_handlers.util import apply_voi_lut

In [13]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

#img = get_img('../input/siim-covid19-detection/test/00188a671292/3eb5a506ccf3/3dcdfc352a06.dcm')
#plt.imshow(img)
#plt.show()

In [18]:
def create_sub_df(test, tst_preds):
    sub_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
    for i in range(len(test)):
        for j in range(len(test)):
            a = test.loc[i,'id'].split('.')[0]
            b = sub_df.loc[j,'id']
            if a==b:
                negative, typical, indeterminate, atypical = str(tst_preds[i][0]),str(tst_preds[i][1]),str(tst_preds[i][2]),str(tst_preds[i][3]),
                sub_df.loc[j,'PredictionString'] = f'negative {negative} 0 0 1 1 typical {typical} 0 0 1 1 indeterminate {indeterminate} 0 0 1 1 atypical {atypical} 0 0 1 1'
    return sub_df

In [19]:
def inference(cfg, is_local, is_ensemble=True):
    if is_local:
        print('Inference on Validation dataset')
    else:
        print('Inference on Test dataset')

    seed_everything(cfg['seed'])
    # folds = StratifiedKFold(n_splits=CFG['fold_num']).split(np.arange(train_study.shape[0]), train_study.label.values)

    if is_local:
        model_dir = f'../models/{cfg["exp_id"]}/'
        train_img_path = '../input/siimcovid19-512-img-png-600-study-png/study/'
        test_img_path = './test/study/'
        train_csv_path = f'../input/siim_covid19_train/{cfg["train_csv_name"]}'
        train = pd.read_csv(train_csv_path)
    else:
        model_dir = f'../input/siim-covid19-studyclass-models/{cfg["exp_id"]}/'
        train_img_path = '../input/siimcovid19-512-img-png-600-study-png/study/'
        test_img_path = './test/study/'
        train_csv_path = f'../input/siim-covid19-train/{cfg["train_csv_name"]}'
        prepare_test_images()
        test = pd.DataFrame()
        test['id'] = list(os.listdir(test_img_path))
        # df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
        # if df.shape[0] == 2477:
        #     fast_sub = True
        #     fast_df = pd.DataFrame(([['00086460a852_study', 'negative 1 0 0 1 1'], 
        #                         ['000c9c05fd14_study', 'negative 1 0 0 1 1'], 
        #                         ['65761e66de9f_image', 'none 1 0 0 1 1'], 
        #                         ['51759b5579bc_image', 'none 1 0 0 1 1']]), 
        #                     columns=['id', 'PredictionString'])
        # else:
        #     fast_sub = False
    
    accs = []
    test_preds_all = 0
    # for fold, (trn_idx, val_idx) in enumerate(folds):
    for idx, fold in enumerate(range(cfg['fold_num'])):
        # train_df = train[train['fold'] != fold]
        # we'll train fold 0 first
#         if fold > 0:
#             break 

        print('Inference fold {} started'.format(fold))

        if is_local:
            valid_df = train[train['fold'] == fold]
            # valid_ = train_study.loc[val_idx,:].reset_index(drop=True)
            #print('valid_',valid_)
            valid_ds = CassavaDataset(valid_df, train_img_path, transforms=get_inference_transforms(), output_label=False, mode='valid')
            val_loader = torch.utils.data.DataLoader(
                valid_ds, 
                batch_size=cfg['valid_bs'],
                num_workers=cfg['num_workers'],
                shuffle=False,
                pin_memory=False,
            )
            data_loader = val_loader
        else:
#             test = pd.DataFrame()
#             test['id'] = list(os.listdir(test_img_path))
            #print('test',test)
            test_ds = CassavaDataset(test, test_img_path, transforms=get_inference_transforms(), output_label=False, mode='test')
            #print('test_ds',test)
            
            tst_loader = torch.utils.data.DataLoader(
                test_ds, 
                batch_size=cfg['valid_bs'],
                num_workers=cfg['num_workers'],
                shuffle=False,
                pin_memory=False,
            )
            data_loader = tst_loader

        device = torch.device(cfg['device'])
#         valid_df.class_id.nunique()
        print(f'Num Class: {cfg["num_classes"]}')
        model = CassvaImgClassifier(cfg['model_arch'], cfg["num_classes"]).to(device)
        

        preds = []
#         ensemble_preds = []
        
        #for epoch in range(CFG['epochs']-3):
        # for i, epoch in enumerate(cfg['used_epochs']):    
        model_path = f'{model_dir}best_acc_{cfg["model_arch"]}_fold{fold}.pt'
        print(f'Loading {model_path}')
        model.load_state_dict(torch.load(model_path))
        
        with torch.no_grad():
            for _ in range(cfg['tta']):
#                 if is_ensemble:
#                     ensemble_preds += [cfg['weights'][0]/sum(cfg['weights'])/cfg['tta']*inference_one_epoch(model=model, data_loader=data_loader, device=device)]
#                 else:
                preds += [cfg['weights'][0]/sum(cfg['weights'])/cfg['tta']*inference_one_epoch(model=model, data_loader=data_loader, device=device)]
#         if is_ensemble:
#             ensemble_preds = np.mean(ensemble_preds, axis=0)
#         else:
        preds = np.mean(preds, axis=0)
    
        if is_ensemble:
            test_preds_all += preds
            
        if is_local:
            acc = (valid_df.class_id.values==np.argmax(preds, axis=1)).mean()
            print('fold {} validation loss = {:.5f}'.format(fold, log_loss(valid_df.class_id.values, preds)))
            print('fold {} validation accuracy = {:.5f}'.format(fold, acc))
            accs.append(acc)
        
        del model
        torch.cuda.empty_cache()
    
    if not is_local:
        if is_ensemble:
            test_preds_all /= cfg['fold_num']
            sumfile = create_sub_df(test, test_preds_all)
        else:
            sumfile = create_sub_df(test, preds)
        sumfile.to_csv('./submission.csv',index=False)
        return sumfile
    else:
        mean_acc = (sum(accs)/cfg["fold_num"])*100.0
        print(f'Mean Accuracy {mean_acc}')

# 4 Main

In [20]:
if __name__ == '__main__':
    cfg = {
        'exp_id': 'exp1',
        'train_csv_name':'train_classification_fold_v1.csv',
        'fold_num': 5,
        'num_classes': 4,
        'seed': 719,
        #'model_arch': 'tf_efficientnet_b7',
        'model_arch': 'tf_efficientnet_b4_ns',
        'img_size': 512,
        'epochs': 100,
        'train_bs': 14,
        'valid_bs': 32,
        'T_0': 10,
        'lr': 1e-6,
        'min_lr': 1e-6,
        'weight_decay':1e-6,
        'num_workers': 4,
        'accum_iter': 2, # suppoprt to do batch accumulation for backprop with effectively larger batch size
        'verbose_step': 1,
        'device': 'cuda:0',
        'tta': 3,
        'used_epochs': [53,55,56,59],
        'weights': [1,1,1,1]
    }

    inference(cfg, is_local, is_ensemble=True)

1it [00:00,  6.42it/s]

Inference on Test dataset


2475it [08:37,  4.78it/s]


Inference fold 0 started
Num Class: 4
Loading ../input/siim-covid19-studyclass-models/exp1/best_acc_tf_efficientnet_b4_ns_fold0.pt


100%|██████████| 38/38 [00:24<00:00,  1.58it/s]


...

100%|██████████| 38/38 [00:23<00:00,  1.61it/s]


Inference fold 3 started
Num Class: 4
Loading ../input/siim-covid19-studyclass-models/exp1/best_acc_tf_efficientnet_b4_ns_fold3.pt


100%|██████████| 38/38 [00:23<00:00,  1.61it/s]


Inference fold 4 started
Num Class: 4
Loading ../input/siim-covid19-studyclass-models/exp1/best_acc_tf_efficientnet_b4_ns_fold4.pt


100%|██████████| 38/38 [00:23<00:00,  1.64it/s]


In [21]:
# fold 0 validation loss = 0.94095
# fold 0 validation accuracy = 0.64162

## YOLOv5 Inference

In [ ]:
import shutil
from tqdm.notebook import tqdm
# import seaborn as sns


In [ ]:
meta = meta[meta['split'] == 'test']
if fast_sub:
    test_df = fast_df.copy()
else:
    test_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
test_df = df[study_len:].reset_index(drop=True) 
meta['image_id'] = meta['image_id'] + '_image'
meta.columns = ['id', 'dim0', 'dim1', 'split']
test_df = pd.merge(test_df, meta, on = 'id', how = 'left')

In [ ]:
def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]

    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height

    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

    return bboxes

In [ ]:
dim = 512 #1024, 256, 'original'
test_dir = f'./{split}/study' #f'/kaggle/tmp/{split}/image'
weights_dir = '/kaggle/input/siim-cov19-yolov5-train/yolov5/runs/train/exp/weights/best.pt'

# import torch
#from IPython.display import Image, clear_output  # to display images

#clear_output()
#print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))


!python detect.py --weights $weights_dir\
                    --img 512\
                    --conf 0.001\
                    --iou 0.5\
                    --source $test_dir\
                    --save-txt --save-conf --exist-ok


image_ids = []
PredictionStrings = []

for file_path in tqdm(glob('runs/detect/exp/labels/*.txt')):
    image_id = file_path.split('/')[-1].split('.')[0]
    w, h = test_df.loc[test_df.id==image_id,['dim1', 'dim0']].values[0]
    f = open(file_path, 'r')
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
    bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 12).astype(str))
    for idx in range(len(bboxes)):
        bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
    image_ids.append(image_id)
    PredictionStrings.append(' '.join(bboxes))


pred_df = pd.DataFrame({'id':image_ids,
                        'PredictionString':PredictionStrings})

In [ ]:
test_df = test_df.drop(['PredictionString'], axis=1)
sub_df = pd.merge(test_df, pred_df, on = 'id', how = 'left').fillna("none 1 0 0 1 1")
sub_df = sub_df[['id', 'PredictionString']]
for i in range(sub_df.shape[0]):
    if sub_df.loc[i,'PredictionString'] == "none 1 0 0 1 1":
        continue
    sub_df_split = sub_df.loc[i,'PredictionString'].split()
    sub_df_list = []
    for j in range(int(len(sub_df_split) / 6)):
        sub_df_list.append('opacity')
        sub_df_list.append(sub_df_split[6 * j + 1])
        sub_df_list.append(sub_df_split[6 * j + 2])
        sub_df_list.append(sub_df_split[6 * j + 3])
        sub_df_list.append(sub_df_split[6 * j + 4])
        sub_df_list.append(sub_df_split[6 * j + 5])
    sub_df.loc[i,'PredictionString'] = ' '.join(sub_df_list)
sub_df['none'] = df_2class['none'] 
for i in range(sub_df.shape[0]):
    if sub_df.loc[i,'PredictionString'] != 'none 1 0 0 1 1':
        sub_df.loc[i,'PredictionString'] = sub_df.loc[i,'PredictionString'] + ' none ' + str(sub_df.loc[i,'none']) + ' 0 0 1 1'
sub_df = sub_df[['id', 'PredictionString']]   
df_study = df_study[:study_len]
df_study = df_study.append(sub_df).reset_index(drop=True)
